## 检索器
***
- 基本检索器设置
- 词法搜索检索器

#### 基本检索器设置
***

In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("test.txt")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(texts, embeddings)

实例化检索器

In [2]:
retriever = vectorstore.as_retriever()

In [3]:
docs = retriever.invoke("deepseek是什么？")

In [4]:
docs

[Document(id='e60ca19a-6b5f-4995-9729-2e2aa6c08782', metadata={'source': 'test.txt'}, page_content='Deepseek R1 是⽀持复杂推理、多模态处理、技术⽂档⽣成的⾼性能通⽤⼤语⾔模型。本⼿册\n为技术团队提供完整的本地部署指南，涵盖硬件配置、国产芯⽚适配、量化⽅案、云端替代⽅\n案及完整671B MoE模型的Ollama部署⽅法。模型 参数规\n模\n计算精\n度\n最低显存需\n求 最低算⼒需求\nDeepSeek-R1 (671B) 671B FP8 ≥890GB\n2*XE9680（16*H20\nGPU）\nDeepSeek-R1-Distill-\n70B\n70B BF16 ≥180GB 4*L20 或 2*H20 GPU\n三、国产芯⽚与硬件适配⽅案\n1. 国内⽣态合作伙伴动态\n企业 适配内容 性能对标（vs\nNVIDIA）\n华为昇\n腾\n昇腾910B原⽣⽀持R1全系列，提供端到端推理优化\n⽅案 等效A100（FP16）\n沐曦\nGPU\nMXN系列⽀持70B模型BF16推理，显存利⽤率提升\n30% 等效RTX 3090\n海光\nDCU 适配V3/R1模型，性能对标NVIDIA A100 等效A100（BF16）\n2. 国产硬件推荐配置\n模型参数 推荐⽅案 适⽤场景\n1.5B 太初T100加速卡 个⼈开发者原型验证\n14B 昆仑芯K200集群 企业级复杂任务推理\n32B 壁彻算⼒平台+昇腾910B集群 科研计算与多模态处理\n12月26日，Deepseek发布了全新系列模型DeepSeek-v3，一夜之间霸榜开源模型，并在性能上和世界顶尖的闭源模型GPT-4o以及 Claude-3.5-Sonnet相提并论。\n\n该模型为MOE架构，大大降低了训练成本，据说训练成本仅600万美元，成本降低10倍，资源运用效率极高。有AI投资机构负责人直言，DeepSeek发布的53页的技术论文是黄金。\n\n那就先让我们看看论文是怎么说的吧，老规矩，先上资源地址：\n\nGithub: GitHub - deepseek-ai/DeepSeek-V3\n\n模型地址：https://huggingface.co/deepseek-ai\

#### BM25也称为Okapi BM25，是信息检索系统中用来估计文档与给定搜索查询的相关性的排名函数。
****


In [5]:
! pip install --upgrade --quiet  rank_bm25


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [6]:
from langchain_community.retrievers import BM25Retriever
vretriever = BM25Retriever.from_texts(["foo", "bar", "world", "hello", "foo bar"])

使用所创建的文档来创建一个新的检索器

In [7]:
from langchain_core.documents import Document

retriever = BM25Retriever.from_documents(
    [
        Document(page_content="foo"),
        Document(page_content="bar"),
        Document(page_content="world"),
        Document(page_content="hello"),
        Document(page_content="foo bar"),
    ]
)

In [8]:
result = retriever.invoke("foo")

In [9]:
result

[Document(metadata={}, page_content='foo'),
 Document(metadata={}, page_content='foo bar'),
 Document(metadata={}, page_content='hello'),
 Document(metadata={}, page_content='world')]

使用预处理器进行增强，在词语级别检索效果比较显著

punkt 是一个句子分割器模型

In [10]:
import nltk

nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/zhangyaohui/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
from nltk.tokenize import word_tokenize

retriever = BM25Retriever.from_documents(
    [
        Document(page_content="foo"),
        Document(page_content="bar"),
        Document(page_content="world"),
        Document(page_content="hello"),
        Document(page_content="foo bar"),
    ],
    k=2,
    preprocess_func=word_tokenize,
)

result = retriever.invoke("bar")
result

[Document(metadata={}, page_content='bar'),
 Document(metadata={}, page_content='foo bar')]